# Text generation with an RNN

### Learning Objectives

- Learn how to generate text using a RNN
- Create training examples and targets for text generation
- Build a RNN model for sequence generation using Keras Subclassing
- Create a text generator and evaluate the output

### Key learnings
- transform chars to numerics using `tf.keras.layers.StringLookup(..., invert=False)`, and vice-versa.
- Building dataset from tensor slices in batches (not to load everything in memory).
- Building a custom sequential model using [`tf.keras.Model`](https://www.tensorflow.org/api_docs/python/tf/keras/Model) to train a model using *teacher forcing*.

This tutorial demonstrates how to generate text using a character-based RNN. You will work with a dataset of Shakespeare's writing from Andrej Karpathy's [The Unreasonable Effectiveness of Recurrent Neural Networks](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Given a sequence of characters from this data ("Shakespear"), train a model to predict the next character in the sequence ("e"). Longer sequences of text can be generated by calling the model repeatedly.

Below is the sample output when the model in this tutorial trained for 30 epochs, and started with the prompt "Q":

<pre>
QUEENE:
I had thought thou hadst a Roman; for the oracle,
Thus by All bids the man against the word,
Which are so weak of care, by old care done;
Your children were in your holy love,
And the precipitation through the bleeding throne.

BISHOP OF ELY:
Marry, and will, my lord, to weep in such a one were prettiest;
Yet now I was adopted heir
Of the world's lamentable day,
To watch the next way with his father with his face?

ESCALUS:
The cause why then we are all resolved more sons.

VOLUMNIA:
O, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, no, it is no sin it should be dead,
And love and pale as any will to that word.

QUEEN ELIZABETH:
But how long have I heard the soul for this world,
And show his hands of life be proved to stand.

PETRUCHIO:
I say he look'd on, if I must be content
To stay him from the fatal of our country's bliss.
His lordship pluck'd from this sentence then for prey,
And then let us twain, being the moon,
were she such a case as fills m
</pre>

While some of the sentences are grammatical, most do not make sense. The model has not learned the meaning of words, but here are some things to consider:

* The model is character-based. When training started, the model did not know how to spell an English word, or that words were even a unit of text.

* The structure of the output resembles a play—blocks of text generally begin with a speaker name, in all capital letters similar to the dataset.

* As demonstrated below, the model is trained on small batches of text (100 characters each), and is still able to generate a longer sequence of text with coherent structure.

## Setup

### Import TensorFlow and other libraries

In [1]:
import os
import warnings

warnings.filterwarnings("ignore")
# os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

In [2]:
import time

import numpy as np
import tensorflow as tf

### Download the Shakespeare dataset

Change the following line to run this code on your own data.

In [3]:
path_to_file = tf.keras.utils.get_file(
    "shakespeare.txt",
    "https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt",
)

### Read the data

First, we'll download the file and then decode.

In [4]:
text = open(path_to_file, "rb").read().decode(encoding="utf-8")
print(f"Length of text: {len(text)} characters")

Length of text: 1115394 characters


Let's take a look at the first 250 characters in text

In [ ]:
print(text[:200])

First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You are all resolved rather to die than to famish?

All:
Resolved. resolved.

First Citizen:
First, you know Caius Marcius is chief enemy to the people.



Let's check to see how many unique characters are in our corpus/document.

In [6]:
# get a list of all unique charcters in the text
vocab = sorted(set(text))
print(f"{len(vocab)} unique characters")

65 unique characters


## Process the text

### Vectorize the text

Before training, you need to convert the strings to a numerical representation. 

Using `tf.keras.layers.StringLookup` layer can convert each character into a numeric ID. It just needs the text to be split into tokens first.

In [7]:
example_texts = ["abcdefg", "xyz"]

# splits into a "list of list", where:
# - 1st index: is the word
# - 2nd index: is the letter
# The object is similar to a list.
chars = tf.strings.unicode_split(example_texts, input_encoding="UTF-8")

Now create the `tf.keras.layers.StringLookup` layer:

In [8]:
# Builds a map that transforms a RaggedTensor of char into IDs
ids_from_chars = tf.keras.layers.StringLookup(
    vocabulary=list(vocab), mask_token=None
)
# convert the example text into IDs
ids = ids_from_chars(chars)
ids

<tf.RaggedTensor [[40, 41, 42, 43, 44, 45, 46], [63, 64, 65]]>

Since the goal of this tutorial is to generate text, it will also be important to invert this representation and recover human-readable strings from it. For this you can use `tf.keras.layers.StringLookup(..., invert=True)`.  

Note: Here instead of passing the original vocabulary generated with `sorted(set(text))` use the `get_vocabulary()` method of the `tf.keras.layers.StringLookup` layer so that the `[UNK]` tokens is set the same way.

In [9]:
chars_from_ids = tf.keras.layers.StringLookup(
    vocabulary=ids_from_chars.get_vocabulary(), invert=True, mask_token=None
)

This layer recovers the characters from the vectors of IDs, and returns them as a `tf.RaggedTensor` of characters:

In [10]:
chars = chars_from_ids(ids)
print(chars)
# Note: you can decode to numpy as 
# chars.numpy().decode("utf-8")


<tf.RaggedTensor [[b'a', b'b', b'c', b'd', b'e', b'f', b'g'], [b'x', b'y', b'z']]>


You can `tf.strings.reduce_join` to join the characters back into strings. 

In [11]:
tf.strings.reduce_join(chars, axis=-1).numpy()

array([b'abcdefg', b'xyz'], dtype=object)

In [12]:
def text_from_ids(ids):
    return tf.strings.reduce_join(chars_from_ids(ids), axis=-1)

### The prediction task

Given a character, or a sequence of characters, what is the most probable next character? This is the task you're training the model to perform. The input to the model will be a sequence of characters, and you train the model to predict the output—the following character at each time step.

Since RNNs maintain an internal state that depends on the previously seen elements, given all the characters computed until this moment, what is the next character?


### Create training examples and targets

Next divide the text into example sequences. Each input sequence will contain `SEQ_LENGTH` characters from the text, which is a *parameter* for the model.

For each input sequence, the corresponding targets contain the same length of text, except shifted one character to the right. For example, if `SEQ_LENGTH = 4` and our text is "Hello". The input sequence would be "Hell", and the target sequence "ello". 

Thia method is called *teacher forcing*: instead of letting the decoder produce all token during training, we force in input the correct token (i.e. the state).

![Encoder-Decoder Architcture Training. In general, the decoder also needs it own input. To improve training, we can employ a method called *teacher forcing*, where we give the decoder the correct previous token as input to generate the next token, rather than what the decoder has generated so far.](docs/images/encoder-decoder-training.png)


First use the `tf.data.Dataset.from_tensor_slices` function to convert the text vector into a stream of character indices.

In [13]:
# Parameters:
SEQ_LENGTH = 100

In [14]:
# Build a Tensor with all numerical IDs
all_ids = ids_from_chars(tf.strings.unicode_split(text, "UTF-8"))
all_ids

<tf.Tensor: shape=(1115394,), dtype=int64, numpy=array([19, 48, 57, ..., 46,  9,  1])>

In [15]:
ids_dataset = tf.data.Dataset.from_tensor_slices(all_ids)
ids_dataset

<_TensorSliceDataset element_spec=TensorSpec(shape=(), dtype=tf.int64, name=None)>

In [16]:
for ids in ids_dataset.take(8):
    print(chars_from_ids(ids).numpy().decode("utf-8"))

F
i
r
s
t
 
C
i


2025-09-27 16:59:21.395275: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


In [17]:
# Note: each input/output pairs uses SEQ_LENGTH+1 characters
n_examples_per_epoch = len(text) // (SEQ_LENGTH + 1) 

The `batch` method lets you easily convert these individual characters to sequences of the desired size.

In [18]:
sequences = ids_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

for seq in sequences.take(1):
    print(chars_from_ids(seq))

tf.Tensor(
[b'F' b'i' b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':'
 b'\n' b'B' b'e' b'f' b'o' b'r' b'e' b' ' b'w' b'e' b' ' b'p' b'r' b'o'
 b'c' b'e' b'e' b'd' b' ' b'a' b'n' b'y' b' ' b'f' b'u' b'r' b't' b'h'
 b'e' b'r' b',' b' ' b'h' b'e' b'a' b'r' b' ' b'm' b'e' b' ' b's' b'p'
 b'e' b'a' b'k' b'.' b'\n' b'\n' b'A' b'l' b'l' b':' b'\n' b'S' b'p' b'e'
 b'a' b'k' b',' b' ' b's' b'p' b'e' b'a' b'k' b'.' b'\n' b'\n' b'F' b'i'
 b'r' b's' b't' b' ' b'C' b'i' b't' b'i' b'z' b'e' b'n' b':' b'\n' b'Y'
 b'o' b'u' b' '], shape=(101,), dtype=string)


2025-09-27 16:59:21.409515: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


It's easier to see what this is doing if you join the tokens back into strings:

In [19]:
for seq in sequences.take(5):
    print(text_from_ids(seq).numpy())

b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
b'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
b"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
b"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
b'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


For training you'll need a dataset of `(input, label)` pairs. Where `input` and 
`label` are sequences. At each time step the input is the current character and the label is the next character. 

Here's a function that takes a sequence as input, duplicates, and shifts it to align the input and label for each timestep:

In [20]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

In [21]:
split_input_target(list("Tensorflow"))

(['T', 'e', 'n', 's', 'o', 'r', 'f', 'l', 'o'],
 ['e', 'n', 's', 'o', 'r', 'f', 'l', 'o', 'w'])

In [22]:
dataset = sequences.map(split_input_target)

In [23]:
for input_example, target_example in dataset.take(1):
    print("Input :", text_from_ids(input_example).numpy())
    print("Target:", text_from_ids(target_example).numpy())

Input : b'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target: b'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


2025-09-27 16:59:21.469081: I tensorflow/core/framework/local_rendezvous.cc:407] Local rendezvous is aborting with status: OUT_OF_RANGE: End of sequence


### Create training batches

You used `tf.data` to split the text into manageable sequences. But before feeding this data into the model, you need to shuffle the data and pack it into batches.

In [24]:
# Batch size
BATCH_SIZE = 64

# Buffer size to shuffle the dataset
# (TF data is designed to work with possibly infinite sequences,
# so it doesn't attempt to shuffle the entire sequence in memory. Instead,
# it maintains a buffer in which it shuffles elements).
BUFFER_SIZE = 10000

dataset = (
    dataset.shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
    .prefetch(tf.data.experimental.AUTOTUNE)
)

dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## Build The Model

This section defines the model as a `keras.Model` subclass (For details see [Making new Layers and Models via subclassing](https://www.tensorflow.org/guide/keras/custom_layers_and_models)). A custom model can allow us to specify specific behavious in training.

### Typical Parameters

In [25]:
# --- 1. Define Key Hyperparameters ---
# These parameters control the architecture and capacity of the RNN.

# VOCAB_SIZE: The total number of unique words/characters/token in your vocabulary.
# This is crucial because the model can only process words it has seen during training.
# It determines the input dimension of the Embedding layer (e.g., if you have 10,000 unique words, this is 10000).
VOCAB_SIZE = len(vocab)

# EMBEDDING_DIM: The size of the vector space in which words/tokens/chars will be embedded.
# Each word is mapped to a dense vector of this dimension. This is a critical parameter that
# allows the model to learn relationships between words. A larger dimension can capture more
# complex relationships but requires more data to train effectively.
# Common values are 64, 128, or 256.
EMBEDDING_DIM = 256

# RNN_UNITS: The number of units (or neurons) in the RNN layer (e.g., LSTM or GRU).
# This defines the dimensionality of the hidden state, which acts as the model's "memory."
# A larger number of units gives the model more capacity to learn complex patterns
# in sequences, but also increases the risk of overfitting.
RNN_UNITS = 1024

# (Optional) MAX_LENGTH: The maximum length of the input sequences. This is only needed if using the
# Sequential API. But as we are building a sequence to sequence model (we are using teacher forcing),
# we won't do this. 
# All text inputs will be padded or truncated to this fixed length. RNNs require
# inputs to have a consistent shape, so this parameter is essential.
MAX_LENGTH = SEQ_LENGTH

### Using tf.kera.Sequential

In [26]:
# Build model (without using teacher forcing). 
# 
# This is a sequence to 1 model: as such, we must specify an input layer (MAX_LENGTH,) that will
# ingest the seuqnece of integers in our vocabulary.

model = tf.keras.Sequential([
    # Input Layer: Defines the shape of the input sequences.
    tf.keras.layers.Input(shape=(MAX_LENGTH,), name="input_layer"),

    # Embedding Layer: This is the first hidden layer.
    # It takes integer-encoded vocabulary and looks up the embedding vector for each word.
    # The output is a 3D tensor of shape: (batch_size, MAX_LENGTH, EMBEDDING_DIM).
    tf.keras.layers.Embedding(
        input_dim=VOCAB_SIZE,
        output_dim=EMBEDDING_DIM,
        name="embedding_layer"
    ),

    # RNN Layer (LSTM): The core of the model for processing sequences.
    # We use LSTM (Long Short-Term Memory) here, as it's generally more powerful
    # than a SimpleRNN because it can learn long-range dependencies.
    # It processes the sequence of word vectors and outputs a single vector representing
    # the entire sequence's context.
    tf.keras.layers.LSTM(
        units=RNN_UNITS,
        name="lstm_layer"
    ),

    # Output Layer: A standard Dense layer for classification.
    # We use a single neuron with a 'sigmoid' activation function for binary
    # classification (outputting a probability between 0 and 1).
    tf.keras.layers.Dense(1, activation='sigmoid', name="output_layer")
])

# The model size is already defined (and hence the no. of parameters)
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding_layer (Embedding)     │ (None, 100, 256)       │        16,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ lstm_layer (LSTM)               │ (None, 1024)           │     5,246,976 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ output_layer (Dense)            │ (None, 1)              │         1,025 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 5,264,641 (20.08 MB)

 Trainable params: 5,264,641 (20.08 MB)

 Non-trainable params: 0 (0.00 B)

### Custom Model with teacher forcing.

See the `CustomRNN` class in [custom_rnn_with_teacher_forcing.py](custom_rnn_with_teacher_forcing.py). The key difference is that we do ot define the input size `MAX_LENGHT`. We let it infer at training time.

In [27]:
from custom_rnn_with_teacher_forcing import CustomRNN

model = CustomRNN(
    # Be sure the vocabulary size matches the `StringLookup` layers.
    vocab_size=len(ids_from_chars.get_vocabulary()),
    embedding_dim=EMBEDDING_DIM,
    rnn_units=RNN_UNITS,
)

In [28]:
# Build the model by passing a sample input shape.
# We pass a tuple of (input_sequence, target_sequence)
# The `None` allows for variable batch size and sequence length.
# model.build(input_shape=[(None, None), (None, None)])

In [29]:
# model.summary()

## Try the model

Now run the model to see that it behaves as expected.

First check the shape of the output:

In [30]:
for input_example_batch, target_example_batch in dataset.take(1):
    example_batch_predictions = model(input_example_batch)
    print(
        example_batch_predictions.shape,
        "# (batch_size, sequence_length, vocab_size)",
    )

(64, 100, 66) # (batch_size, sequence_length, vocab_size)


In the above example the sequence length of the input is `100` but the model can be run on inputs of any length:

In [31]:
model.summary()

Model: "custom_rnn"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (64, 100, 256)         │        16,896 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ gru (GRU)                       │ ((64, 100, 1024), (64, │     3,938,304 │
│                                 │ 1024))                 │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (64, 100, 66)          │        67,650 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,022,850 (15.35 MB)

 Trainable params: 4,022,850 (15.35 MB)

 Non-trainable params: 0 (0.00 B)

To get actual predictions from the model you need to sample from the output distribution, to get actual character indices. This distribution is defined by the logits over the character vocabulary.

Note: It is important to _sample_ from this distribution as taking the _argmax_ of the distribution can easily get the model stuck in a loop.

Try it for the first example in the batch:

In [32]:
sampled_indices = tf.random.categorical(
    example_batch_predictions[0], num_samples=1
)
sampled_indices = tf.squeeze(sampled_indices, axis=-1).numpy()

This gives us, at each timestep, a prediction of the next character index:

In [33]:
sampled_indices

array([45, 47,  5, 30, 65, 47, 28, 11,  9, 37, 42, 37, 29, 29, 15, 36, 55,
       51, 14, 57, 52,  6, 19,  2, 21, 48, 15, 50, 20, 54, 23, 45, 63, 14,
       22,  5, 34, 52, 54, 59, 16, 24, 17, 45, 61, 12, 56, 23, 30, 56,  9,
       40, 23, 51,  7, 60, 21, 16, 20, 14, 47, 42,  5, 65, 12, 39, 51, 25,
       13, 33, 15, 46, 36, 48,  8, 15,  2, 25, 22, 28, 10, 26,  6,  6,  1,
       26,  5, 53, 62, 19, 24, 17, 15, 47, 59, 15, 62, 43,  7, 26])

Decode these to see the text predicted by this untrained model:

In [34]:
print("Input:\n", text_from_ids(input_example_batch[0]).numpy())
print()
print("Next Char Predictions:\n", text_from_ids(sampled_indices).numpy())

Input:
 b'nsed as thought on.\n\nCAMILLO:\nWell, my lord,\nIf you may please to think I love the king\nAnd through '

Next Char Predictions:
 b"fh&QzhO:.XcXPPBWplArm'F HiBkGoJfxAI&UmotCKDfv;qJQq.aJl,uHCGAhc&z;ZlL?TBgWi-B LIO3M''\nM&nwFKDBhtBwd,M"


## Train the model

At this point the problem can be treated as a standard classification problem. Given the previous RNN state, and the input this time step, predict the class of the next character.

#### Attach an optimizer, and a loss function
The standard `tf.keras.losses.sparse_categorical_crossentropy` loss function works in this case because it is applied across the last dimension of the predictions.

Because your model returns logits, you need to set the `from_logits` flag.


In [35]:
# TODO - add a loss function here
loss = tf.losses.SparseCategoricalCrossentropy(from_logits=True)

In [36]:
example_batch_mean_loss = loss(target_example_batch, example_batch_predictions)
print(
    "Prediction shape: ",
    example_batch_predictions.shape,
    " # (batch_size, sequence_length, vocab_size)",
)
print("Mean loss:        ", example_batch_mean_loss)

Prediction shape:  (64, 100, 66)  # (batch_size, sequence_length, vocab_size)
Mean loss:         tf.Tensor(4.1889844, shape=(), dtype=float32)


A newly initialized model shouldn't be too sure of itself, the output logits should all have similar magnitudes. To confirm this you can check that the exponential of the mean loss is approximately equal to the vocabulary size. A much higher loss means the model is sure of its wrong answers, and is badly initialized:

In [37]:
tf.exp(example_batch_mean_loss).numpy()

np.float32(65.95577)

Configure the training procedure using the `tf.keras.Model.compile` method. Use `tf.keras.optimizers.Adam` with default arguments and the loss function.

In [38]:
model.compile(optimizer="adam", loss=loss)

### Configure checkpoints & Train

Use a `tf.keras.callbacks.ModelCheckpoint` to ensure that checkpoints are saved during training:

In [39]:
LOAD_EXISTING_MODEL = True
# Directory where the checkpoints will be saved
checkpoint_dir = "./training_checkpoints"

if LOAD_EXISTING_MODEL:

    # --- Instantiate the Model ---
    print("Instantiating the model structure...")
    model = CustomRNN(
        vocab_size=VOCAB_SIZE,
        embedding_dim=EMBEDDING_DIM,
        rnn_units=RNN_UNITS
    )

    # --- Load the Weights ---
    print("Loading the trained model weights...")
    
    # The weights are saved in H5 format, so we point directly to the file.
    weights_path = './training_checkpoints/custom_rnn_weights.weights.h5'
    
    # Before loading weights, the model must be "built".
    # Calling it on a sample input is a robust way to do this.
    # The batch size of 1 is for text generation.
    model.build(tf.TensorShape([1, None]))
    
    model.load_weights(weights_path)
    print("Model weights loaded successfully.")


else:
    # Name of the checkpoint files
    checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}.weights.h5")

    checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
        filepath=checkpoint_prefix, save_weights_only=True
    )

    EPOCHS = 10
    history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

    model.save_weights(os.path.join(checkpoint_dir, 'custom_rnn_weights.weights.h5'))
    print(f"Model weights saved to {checkpoint_dir}/custom_rnn_weights.weights.h5")

Instantiating the model structure...
Loading the trained model weights...
Model weights loaded successfully.


#### Generate text

The simplest way to generate text with this model is to run it in a loop, and keep track of the model's internal state as you execute it.

Each time you call the model you pass in some text and an internal state. The model returns a prediction for the next character and its new state. Pass the prediction and state back in to continue generating text.


The following makes a single step prediction:

In [ ]:
# class OneStep(tf.keras.Model):
#     def __init__(self, model, chars_from_ids, ids_from_chars, temperature=1.0):
#         super().__init__()
#         self.temperature = temperature
#         self.model = model
#         self.chars_from_ids = chars_from_ids
#         self.ids_from_chars = ids_from_chars

#         # Create a mask to prevent "[UNK]" from being generated.
#         skip_ids = self.ids_from_chars(["[UNK]"])[:, None]
#         sparse_mask = tf.SparseTensor(
#             # Put a -inf at each bad index.
#             values=[-float("inf")] * len(skip_ids),
#             indices=skip_ids,
#             # Match the shape to the vocabulary
#             dense_shape=[len(ids_from_chars.get_vocabulary())],
#         )
#         self.prediction_mask = tf.sparse.to_dense(sparse_mask)

#     @tf.function
#     def generate_one_step(self, inputs, states=None):
#         # Convert strings to token IDs.
#         input_chars = tf.strings.unicode_split(inputs, "UTF-8")
#         input_ids = self.ids_from_chars(input_chars).to_tensor()

#         # Run the model.
#         # predicted_logits.shape is [batch, char, next_char_logits]
#         predicted_logits, states = self.model(
#             inputs=input_ids, states=states, return_state=True
#         )
#         # Only use the last prediction.
#         predicted_logits = predicted_logits[:, -1, :]
#         predicted_logits = predicted_logits / self.temperature
#         # Apply the prediction mask: prevent "[UNK]" from being generated.
#         predicted_logits = predicted_logits + self.prediction_mask

#         # Sample the output logits to generate token IDs.
#         predicted_ids = tf.random.categorical(predicted_logits, num_samples=1)
#         predicted_ids = tf.squeeze(predicted_ids, axis=-1)

#         # Convert from token ids to characters
#         predicted_chars = self.chars_from_ids(predicted_ids)

#         # Return the characters and model state.
#         return predicted_chars, states
    
#     one_step_model = OneStep(model, chars_from_ids, ids_from_chars)

Run it in a loop to generate some text. Looking at the generated text, you'll see the model knows when to capitalize, make paragraphs and imitates a Shakespeare-like writing vocabulary. With the small number of training epochs, it has not yet learned to form coherent sentences.

In [46]:
# start = time.time()
# states = None
# next_char = tf.constant(["ROMEO:"])
# result = [next_char]

# for n in range(1000):
#     next_char, states = one_step_model.generate_one_step(
#         next_char, states=states
#     )
#     result.append(next_char)

# result = tf.strings.join(result)
# end = time.time()
# print(result[0].numpy().decode("utf-8"), "\n\n" + "_" * 80)
# print("\nRun time:", end - start)

In [47]:
from custom_rnn_with_teacher_forcing import TextGenerator
print("\n--- Generating Text ---")
generator = TextGenerator(model, chars_to_ids, ids_to_chars)
generated_text = generator.generate(start_string="JULIET:", num_generate=500)
print(generated_text)

ImportError: cannot import name 'TextGenerator' from 'custom_rnn_with_teacher_forcing' (/Users/salvatore.maraniello/git/utils/snip-py/tf-keras-examples/sequential-models/custom_rnn_with_teacher_forcing.py)

The easiest thing you can do to improve the results is to train it for longer (try `EPOCHS = 30`).

You can also experiment with a different start string, try adding another RNN layer to improve the model's accuracy, or adjust the temperature parameter to generate more or less random predictions.

If you want the model to generate text *faster* the easiest thing you can do is batch the text generation. In the example below the model generates 5 outputs in about the same time it took to generate 1 above. 

In [ ]:
# start = time.time()
# states = None
# next_char = tf.constant(["ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:", "ROMEO:"])
# result = [next_char]

# for n in range(1000):
#     next_char, states = one_step_model.generate_one_step(
#         next_char, states=states
#     )
#     result.append(next_char)

# result = tf.strings.join(result)
# end = time.time()
# for s in result:
#     print(s.numpy().decode("utf-8"))
# print("\n\n" + "_" * 80)
# print("\nRun time:", end - start)

## Export the generator

This single-step model can easily be [saved and restored](https://www.tensorflow.org/guide/saved_model), allowing you to use it anywhere a `tf.saved_model` is accepted.

In [ ]:
tf.saved_model.save(one_step_model, "one_step")
one_step_reloaded = tf.saved_model.load("one_step")

In [ ]:
states = None
next_char = tf.constant(["ROMEO:"])
result = [next_char]

for n in range(100):
    next_char, states = one_step_reloaded.generate_one_step(
        next_char, states=states
    )
    result.append(next_char)

print(tf.strings.join(result)[0].numpy().decode("utf-8"))

## Advanced: Customized Training

The above training procedure is simple, but does not give you much control.
It uses teacher-forcing which prevents bad predictions from being fed back to the model, so the model never learns to recover from mistakes.

So now that you've seen how to run the model manually next you'll implement the training loop. This gives a starting point if, for example, you want to implement _curriculum  learning_ to help stabilize the model's open-loop output.

The most important part of a custom training loop is the train step function.

Use `tf.GradientTape` to track the gradients. You can learn more about this approach by reading the [eager execution guide](https://www.tensorflow.org/guide/eager).

The basic procedure is:

1. Execute the model and calculate the loss under a `tf.GradientTape`.
2. Calculate the updates and apply them to the model using the optimizer.

In [ ]:
class CustomTraining(MyModelTeacherForcing):
    @tf.function
    def train_step(self, inputs):
        inputs, labels = inputs
        with tf.GradientTape() as tape:
            predictions = self(inputs, training=True)
            loss = self.loss(labels, predictions)
        grads = tape.gradient(loss, model.trainable_variables)
        self.optimizer.apply_gradients(zip(grads, model.trainable_variables))

        return {"loss": loss}

The above implementation of the `train_step` method follows [Keras' `train_step` conventions](https://www.tensorflow.org/guide/keras/customizing_what_happens_in_fit). This is optional, but it allows you to change the behavior of the train step and still use keras' `Model.compile` and `Model.fit` methods.

In [ ]:
model = CustomTraining(
    vocab_size=len(ids_from_chars.get_vocabulary()),
    EMBEDDING_DIM=EMBEDDING_DIM,
    RNN_UNITS=RNN_UNITS,
)

In [ ]:
model.compile(
    optimizer=tf.keras.optimizers.Adam(),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
)

In [ ]:
model.fit(dataset, epochs=1)

Or if you need more control, you can write your own complete custom training loop:

In [ ]:
EPOCHS = 10

mean = tf.metrics.Mean()

for epoch in range(EPOCHS):
    start = time.time()

    mean.reset_states()
    for batch_n, (inp, target) in enumerate(dataset):
        logs = model.train_step([inp, target])
        mean.update_state(logs["loss"])

        if batch_n % 50 == 0:
            template = (
                f"Epoch {epoch+1} Batch {batch_n} Loss {logs['loss']:.4f}"
            )
            print(template)

    # saving (checkpoint) the model every 5 epochs
    if (epoch + 1) % 5 == 0:
        model.save_weights(checkpoint_prefix.format(epoch=epoch))

    print()
    print(f"Epoch {epoch+1} Loss: {mean.result().numpy():.4f}")
    print(f"Time taken for 1 epoch {time.time() - start:.2f} sec")
    print("_" * 80)

model.save_weights(checkpoint_prefix.format(epoch=epoch))